In [ ]:
!pip install transformers seqeval[gpu]
!pip install duckling
!pip install --force-reinstall JPype1==0.6.3
!pip install datetime

!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip3 install pandas
!pip3 install numpy
!pip3 install sklearn
!pip3 install tqdm
!pip install pattern

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=0f9322efd189f0be76aadc8f2ae4282528b5911da4cde737fe08292919dcc1cd
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 36.2 MB/s eta 0:00:0

In [ ]:
import pandas as pd
import numpy as np
import torch
import json
import os
import pickle
import torch
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from datetime import datetime
from dateutil.relativedelta import relativedelta, MO
from datetime import timedelta
from duckling import DucklingWrapper
parser = DucklingWrapper()

from torch import nn
from torch.optim import Adam
from transformers import GPT2Model, GPT2Tokenizer
from tqdm import tqdm
from pattern.text.en import singularize

In [ ]:
# PARAMS
MAX_LEN = 128
device = 'cpu'
ids_to_labels ={0: 'O',
 1: 'B-DisplayColumns',
 2: 'B-AggColumns',
 3: 'I-DisplayColumns',
 4: 'B-FilterColumns',
 5: 'I-AggColumns',
 6:'B-date-period',
 7: 'I-FilterColumns',
 8: 'I-date-period',
 9: 'B-DerivedMetricsColumns'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pwd

'/content'

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/Chatbot_github/

/content/drive/My Drive/Colab Notebooks/Chatbot_github


### Entity

In [ ]:
# file reading
with open("model/Entity_model.pkl", "rb") as f:
  model = pickle.load(f)
with open("model/Entity_tokenizer.pkl", "rb") as f:
  tokenizer = pickle.load(f)

In [ ]:
# Entity Mapping TREE
# -*- coding: utf-8 -*-
"""
Created on Tue Jun  6 16:06:11 2023

@author: kaustubh.keshav
"""

files = [f for f in os.listdir('chatbot_pepsico/entities/') if '_en' in f]
all_entities = []
# keyword_mapping = {}
for file in files:
    with open(f"chatbot_pepsico/entities/{file}", "rb") as f:
        mapping = json.load(f)
        all_entities.extend(mapping)
        # update_dict = {j['value']:file.split("_")[0] for j in mapping}
        # keyword_mapping.update(update_dict)

tree_dict = {}
for dic in all_entities:
    tag = dic['value']
    synonyms = dic['synonyms']

    for synonym in synonyms:
 #       print(tree_dict)
        symbols = synonym.split(" ")
        root = symbols[0]
        root = singularize(root)
        if root in tree_dict:
            curr_level = tree_dict[root]
        else:
            tree_dict[root] = {}
            curr_level = tree_dict[root]
#        print(curr_level, root)

        for symbol in symbols[1:]:
            symbol = singularize(symbol)
            if symbol in curr_level:
                curr_level = curr_level[symbol]
            else:
                curr_level[symbol] = {}
                # parent = curr_level
                curr_level = curr_level[symbol]

        curr_level['leaf'] = tag

In [ ]:
# making possible entity mapping
tags_dic = {'DisplayColumns_entries_en.json':'DisplayColumns', 'AggColumns_entries_en.json':'AggColumns', 'DerivedMetricsColumns_entries_en.json':'DerivedMetricsColumns', 'FilterColumns_entries_en.json':'FilterColumns',
            'forecast_entries_en.json':'forecast', 'function_entries_en.json':'function', 'mathematical_filter_entries_en.json':'mathematical_filter', 'n_rows_entries_en.json':'n_rows', 'y_col_entries_en.json':'y_col'}

all_entities = []
entity_mapping = pd.DataFrame()

for file in tags_dic.keys():
    with open(f"chatbot_pepsico/entities/{file}", "rb") as f:
        mapping = json.load(f)
        df1 = pd.DataFrame(mapping)
        df1['entity'] = tags_dic[file]
        entity_mapping = pd.concat([entity_mapping, df1[['value', 'entity']]], ignore_index=True)

entity_mapping_dic = {}
for val in list(entity_mapping.value.unique()):
  entity_mapping_dic[val] = list(entity_mapping[entity_mapping.value == val]['entity'])

In [ ]:
def word_correction(sentence):
  node = tree_dict
  # for word in sentence.split(" "):
  i = 0
  curr_i = 0
  words = sentence.split(" ")
  token_entity_map = {}
  tokens = []

  while curr_i < len(words) and i < len(words):
      word = words[i]
      try:
          node = node[word]
          i += 1
          tokens.append(word)
      except:
          try:
              token_entity_map[' '.join(tokens)] = node['leaf']
              node = tree_dict
              curr_i = i #here is the change. earlier it was curr_i = i + 1
              i = curr_i

          except:
              node = tree_dict
              i = curr_i + 1
              curr_i = i

          tokens = []

  return token_entity_map

In [ ]:
def response(sentence):

  inputs = tokenizer(sentence.split(),
                      is_split_into_words=True,
                      return_offsets_mapping=True,
                      padding='max_length',
                      truncation=True,
                      max_length=MAX_LEN,
                      return_tensors="pt")

  # move to gpu
  ids = inputs["input_ids"].to(device)
  mask = inputs["attention_mask"].to(device)
  # forward pass
  outputs = model(ids, attention_mask=mask)
  logits = outputs[0]

  active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
  flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

  tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
  token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
  wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

  prediction = []
  for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
    #only predictions on first word pieces are important
    if mapping[0] == 0 and mapping[1] != 0:
      prediction.append(token_pred[1])
    else:
      continue

  return ", ".join(prediction)
#  print(sentence.split())
#  print(prediction)

In [ ]:
def return_date(text, parser):
  output = parser.parse_time(text)
  date = []
  for t in output:
    val = t['value']['value']
    date_dict = {}
    if isinstance(val, dict):
      date_dict['endDate'] = val['to']
      date_dict['startDate'] = val['from']
    if ~isinstance(val, dict) & ("grain" in t['value']):
      grain = t['value']['grain']
      date_dict['startDate'] = val
      if grain == 'year':
        new_date = datetime.strptime(val,'%Y-%m-%dT%H:%M:%S.%fZ') + relativedelta(years=1)
        new_date = new_date.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        date_dict['endDate'] = new_date
      if grain == 'month':
        new_date = datetime.strptime(val,'%Y-%m-%dT%H:%M:%S.%fZ') + relativedelta(months=1)
        new_date = new_date.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        date_dict['endDate'] = new_date

    date.append(date_dict)
  return date

In [ ]:
# Mapping dictionary
# layer 1

keys = list(tags_dic.values()) + ['date-period']

def Response_entity(sent):
  map_dictionary = {}
  sent0 = sent
  sent = sent.replace('?', '').strip()

  values = []
  word = sent.split(' ')
  tags = response(sent).split(', ')

  sent = (' ').join([singularize(w) for w in sent.lower().split()])
  sent = sent.lower()
  token_entity_map = word_correction(sent+' ')

#########
  correction_dic = {}
  idx = []
  idx_ = []
  for key in token_entity_map.keys():
    index = sent.replace(key, key.replace(' ','_')).split().index(key.replace(' ','_'))
    if 'date-period' not in tags[index]:
      idx_.append(index)
      try:
        l = len(entity_mapping_dic[token_entity_map[key]])
      except:
        l = 0
      if l != 1:
        correction_dic[key] = tags[index]
      elif l == 1:
        correction_dic[key] = 'B-' + entity_mapping_dic[token_entity_map[key]][0]

      idx = idx+list(range(index,index+len(key.split())))

  word_upd = [ele for id, ele in enumerate(word) if id not in idx]
  tags_upd = [ele for id, ele in enumerate(tags) if id not in idx]

  for key, val in correction_dic.items():
    word_upd.append(token_entity_map[key])
    tags_upd.append(val)
#########

# layer 2
  for tag in keys:
    word_=[]
    tag_=[]
    for i in range(len(word_upd)):
      if tag in tags_upd[i]:
          word_.append(word_upd[i])
          tag_.append(tags_upd[i])

    ar_ = []
    j=0
    while j < len(word_):
      if 'B-' in tag_[j]:
        ar_.append(word_[j])
      elif 'I-' in tag_[j]:
        ar_[-1]=ar_[-1]+' '+word_[j]
      j=j+1

    values.append(ar_)

  map_dic = dict(zip(keys,values))
  map_dic['date-period'] = return_date((' ').join(map_dic['date-period']), parser)

#  map_dictionary[sent0] = map_dic

  return map_dic

In [ ]:
Response_entity('what are top 5 contributors for sale in us in 2022')

{'DisplayColumns': [],
 'AggColumns': [],
 'DerivedMetricsColumns': [],
 'FilterColumns': ['Market:US'],
 'forecast': [],
 'function': ['contributors'],
 'mathematical_filter': [],
 'n_rows': ['top'],
 'y_col': ['Gross Sales ($)'],
 'date-period': [{'startDate': '2022-01-01T00:00:00.000Z',
   'endDate': '2023-01-01T00:00:00.000000Z'}]}

### Intent

In [ ]:
class SimpleGPT2SequenceClassifier(nn.Module):
    def __init__(self, hidden_size: int, num_classes:int ,max_seq_len:int, gpt_model_name:str):
        super(SimpleGPT2SequenceClassifier,self).__init__()
        self.gpt2model = GPT2Model.from_pretrained(gpt_model_name)
        self.fc1 = nn.Linear(hidden_size*max_seq_len, num_classes)


    def forward(self, input_id, mask):
        """
        Args:
                input_id: encoded inputs ids of sent.
        """
        gpt_out, _ = self.gpt2model(input_ids=input_id, attention_mask=mask, return_dict=False)
        batch_size = gpt_out.shape[0]
        linear_output = self.fc1(gpt_out.view(batch_size,-1))
        return linear_output

In [ ]:
# load trained model
with open("model/Intent_model.pkl", "rb") as f:
  model_intent = pickle.load(f)

with open("model/Intent_tokenizer.pkl", "rb") as f:
  tokenizer_intent = pickle.load(f)
tokenizer_intent.padding_side = "left"
tokenizer_intent.pad_token = tokenizer_intent.eos_token

In [ ]:
def Response_intent(sent):
  fixed_text = " ".join(sent.lower().split())

  model_input = tokenizer_intent(fixed_text, padding='max_length', max_length=128, truncation=True, return_tensors="pt")

  mask = model_input['attention_mask'].cpu()
  input_id = model_input["input_ids"].squeeze(1).cpu()

  output = model_intent(input_id, mask)
  prob = torch.nn.functional.softmax(output, dim=1)[0]

  labels_map = {0: 'Aggregation',
  1: 'compare_trend',
  2: 'Default Welcome Intent',
  3: 'derived_metrics',
  4: 'no_comp_trend',
  5: 'no_trend_compare',
  6: 'simple_graphs'}

  pred_label = labels_map[output.argmax(dim=1).item()]
  return pred_label

In [ ]:
Response_intent('sales of markets across years')

'Aggregation'

#### Returning Intent and Entity for a query:

In [ ]:
def Model_Response(sent):
  Model_Response = {}
  Model_Response['queryText'] = sent
  Model_Response['parameters'] = Response_entity(sent)
  Model_Response['intent'] = Response_intent(sent)

  return Model_Response

In [ ]:
Model_Response('sales of markets across years')

{'queryText': 'sales of markets across years',
 'parameters': {'DisplayColumns': ['Year'],
  'AggColumns': ['Market'],
  'DerivedMetricsColumns': [],
  'FilterColumns': [],
  'forecast': [],
  'function': [],
  'mathematical_filter': [],
  'n_rows': [],
  'y_col': ['Gross Sales ($)'],
  'date-period': []},
 'intent': 'Aggregation'}